# സെമാന്റിക് കർണലിൽ ഏജന്റ് സ്ക്രാച്ച്പാഡുമായി ചാറ്റ് ഹിസ്റ്ററി കുറയ്ക്കൽ

ഈ നോട്ട്‌ബുക്ക്, സെമാന്റിക് കർണലിന്റെ ചാറ്റ് ഹിസ്റ്ററി കുറയ്ക്കൽ സവിശേഷതയും, സംഭാഷണങ്ങളിൽ പ്ര_CONTEXT_ നിലനിർത്തുന്നതിനുള്ള ഏജന്റ് സ്ക്രാച്ച്പാഡും എങ്ങനെ ഉപയോഗിക്കാമെന്ന് കാണിക്കുന്നു. ദീർഘമായ സംഭാഷണങ്ങൾ കൈകാര്യം ചെയ്യുന്നതിനുള്ള ടോക്കൺ പരിധി ലംഘിക്കാതെ പ്രവർത്തിക്കുന്ന കാര്യക്ഷമമായ AI ഏജന്റുകൾ നിർമ്മിക്കുന്നതിന് ഇത് അനിവാര്യമാണ്.

## നിങ്ങൾ പഠിക്കുന്നതെന്താണ്:
1. **ചാറ്റ് ഹിസ്റ്ററി കുറയ്ക്കൽ**: ടോക്കൺ ഉപയോഗം നിയന്ത്രിക്കുന്നതിന് സംഭാഷണ ചരിത്രം സ്വയം സംഗ്രഹിക്കുന്നതെങ്ങനെ
2. **ഏജന്റ് സ്ക്രാച്ച്പാഡ്**: ഉപയോക്തൃ ഇഷ്ടങ്ങളും പൂർത്തിയാക്കിയ പ്രവർത്തനങ്ങളും ട്രാക്ക് ചെയ്യുന്നതിനുള്ള സ്ഥിരമായ മെമ്മറി സിസ്റ്റം
3. **ടോക്കൺ ഉപയോഗം ട്രാക്കിംഗ്**: ഹിസ്റ്ററി കുറയ്ക്കലോടും കൂടാതെ ടോക്കൺ ഉപയോഗം എങ്ങനെ മാറുന്നു എന്ന് നിരീക്ഷിക്കുക

## ആവശ്യകതകൾ:
- പരിസ്ഥിതി വേരിയബിളുകൾ ക്രമീകരിച്ച Azure OpenAI സജ്ജീകരണം
- മുൻ പാഠങ്ങളിൽ നിന്ന് അടിസ്ഥാന ഏജന്റ് ആശയങ്ങൾ മനസ്സിലാക്കുക


## ആവശ്യമായ പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുക


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## ഏജന്റ് സ്ക്രാച്ച്പാഡ് മനസ്സിലാക്കുക

### ഏജന്റ് സ്ക്രാച്ച്പാഡ് എന്താണ്?

**ഏജന്റ് സ്ക്രാച്ച്പാഡ്** എന്നത് ഏജന്റുകൾ ഉപയോഗിക്കുന്ന ഒരു സ്ഥിരമായ മെമ്മറി സിസ്റ്റമാണ്, ഇത്:
- **പൂർത്തിയാക്കിയ ജോലികൾ ട്രാക്ക് ചെയ്യുക**: ഉപയോക്താവിനായി എന്താണ് ചെയ്തതെന്ന് രേഖപ്പെടുത്തുക
- **ഉപയോക്തൃ ഇഷ്ടങ്ങൾ സംഭരിക്കുക**: ഇഷ്ടങ്ങൾ, അനിഷ്ടങ്ങൾ, ആവശ്യങ്ങൾ എന്നിവ ഓർക്കുക
- **സന്ദർഭം നിലനിർത്തുക**: സംഭാഷണങ്ങളിൽ പ്രധാനപ്പെട്ട വിവരങ്ങൾ ലഭ്യമാക്കുക
- **ആവർത്തനങ്ങൾ കുറയ്ക്കുക**: ഒരേ ചോദ്യങ്ങൾ വീണ്ടും വീണ്ടും ചോദിക്കുന്നത് ഒഴിവാക്കുക

### ഇത് എങ്ങനെ പ്രവർത്തിക്കുന്നു:
1. **റൈറ്റിംഗ് പ്രവർത്തനങ്ങൾ**: ഏജന്റ് പുതിയ വിവരങ്ങൾ പഠിച്ചതിന് ശേഷം സ്ക്രാച്ച്പാഡ് അപ്ഡേറ്റ് ചെയ്യുന്നു
2. **റീഡ് പ്രവർത്തനങ്ങൾ**: തീരുമാനങ്ങൾ എടുക്കുമ്പോൾ ഏജന്റ് സ്ക്രാച്ച്പാഡ് പരിശോധിക്കുന്നു
3. **സ്ഥിരത**: ചാറ്റ് ചരിത്രം കുറച്ചാലും വിവരങ്ങൾ നിലനിൽക്കുന്നു

ഇത് സംഭാഷണ ചരിത്രത്തെ पूരിപ്പിക്കുന്ന ഏജന്റിന്റെ വ്യക്തിഗത നോട്ട്‌ബുക്കിനെപ്പോലെയാണ്.


## പരിസ്ഥിതി കോൺഫിഗറേഷൻ


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## ഏജന്റ് സ്ക്രാച്ച്പാഡ് പ്ലഗിൻ സൃഷ്ടിക്കുക

ഈ പ്ലഗിൻ ഏജന്റിന് സ്ഥിരമായ സ്ക്രാച്ച്പാഡ് ഫയലിൽ വായിക്കാനും എഴുതാനും അനുവദിക്കുന്നു.


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## ചാറ്റ് ഹിസ്റ്ററി റിഡ്യൂസർ ആരംഭിക്കുക

ChatHistorySummarizationReducer ചാറ്റ് ചരിത്രം ഒരു പരിധി കവിയുമ്പോൾ അത് സ്വയമേവ സംഗ്രഹിക്കുന്നു.


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## അവധിക്കാല പദ്ധതിയിടുന്ന ഏജന്റ് സൃഷ്ടിക്കുക

ഈ ഏജന്റ് ഉപയോക്താക്കൾക്ക് അവധിക്കാലം പദ്ധതിയിടാൻ സഹായിക്കും, സ്ക്രാച്ച്പാഡ് വഴി പ്ര_CONTEXT_ നിലനിർത്തും.


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immedi ately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

## പ്രദർശനത്തിനും ടോക്കൺ ട്രാക്കിംഗിനുമുള്ള സഹായി ഫംഗ്ഷനുകൾ


In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## അവധിക്കാല പദ്ധതിയിടൽ സംഭാഷണം നടത്തുക

ഇപ്പോൾ ഒരു പൂർണ്ണ സംഭാഷണം നടത്താം, ഇത് കാണിക്കുന്നു:
1. പ്രാരംഭ പദ്ധതിയിടൽ അഭ്യർത്ഥന
2. ഇഷ്ടങ്ങൾ ശേഖരിക്കൽ
3. യാത്രാ പദ്ധതി സൃഷ്ടിക്കൽ
4. സ്ഥലം മാറ്റം
5. ചാറ്റ് ചരിത്രം കുറയ്ക്കൽ
6. സ്ക്രാച്ച്പാഡ് ഉപയോഗം


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## ഫലങ്ങൾ വിശകലനം ചെയ്യുന്നു

നമ്മുടെ സംഭാഷണത്തിനിടെ എന്താണ് സംഭവിച്ചതെന്ന് വിശകലനം ചെയ്യാം:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## പ്രധാനപ്പെട്ട കാര്യങ്ങൾ

### 1. ചാറ്റ് ചരിത്രം ചുരുക്കൽ
- **സ്വയമേവ സജീവമാക്കൽ**: സന്ദേശങ്ങളുടെ എണ്ണം പരിധി കടക്കുമ്പോൾ ചുരുക്കൽ സജീവമാകും
- **ടോക്കൺ ലാഭം**: ചുരുക്കലിന് ശേഷം ടോക്കൺ ഉപയോഗത്തിൽ വലിയ കുറവ്
- **സന്ദർഭം സംരക്ഷിക്കൽ**: പ്രധാനപ്പെട്ട വിവരങ്ങൾ ചുരുക്കത്തിൽ നിലനിർത്തുന്നു

### 2. ഏജന്റ് സ്ക്രാച്ച്പാഡ് പ്രയോജനങ്ങൾ
- **സ്ഥിരമായ ഓർമ്മ**: ഉപയോക്തൃ മുൻഗണനകൾ ചരിത്ര ചുരുക്കലിന് ശേഷവും നിലനിൽക്കും
- **ടാസ്ക് ട്രാക്കിംഗ്**: ഏജന്റ് പൂർത്തിയാക്കിയ ജോലികളുടെ രേഖ സൂക്ഷിക്കുന്നു
- **മികച്ച അനുഭവം**: മുൻഗണനകൾ ആവർത്തിച്ച് പറയേണ്ട ആവശ്യമില്ല

### 3. ടോക്കൺ ഉപയോഗ പാറ്റേണുകൾ
- **രേഖീയ വളർച്ച**: ഓരോ സന്ദേശത്തോടും കൂടെ ടോക്കണുകളുടെ എണ്ണം വർധിക്കുന്നു
- **വലിയ കുറവ്**: ചുരുക്കൽ ടോക്കൺ എണ്ണത്തിൽ ഗണ്യമായ കുറവ് വരുത്തുന്നു
- **സ്ഥിരമായ സംഭാഷണങ്ങൾ**: പരിധിക്കുള്ളിൽ ദീർഘകാല ഇടപെടലുകൾ സാധ്യമാക്കുന്നു


## ക്ലീൻഅപ്പ്

ഈ ഡെമോയ്ക്കിടെ സൃഷ്ടിച്ച സ്ക്രാച്ച്പാഡ് ഫയൽ ക്ലീൻ ചെയ്യുക:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# സംഗ്രഹം

അഭിനന്ദനങ്ങൾ! നിങ്ങൾ പുരോഗമിച്ച കോൺടെക്സ്റ്റ് മാനേജ്മെന്റ് കഴിവുകളുള്ള ഒരു AI ഏജന്റിനെ വിജയകരമായി നടപ്പിലാക്കി:

## നിങ്ങൾ പഠിച്ചതെന്താണ്:
- **ചാറ്റ് ചരിത്രം ചുരുക്കൽ**: ടോക്കൺ പരിധി നിയന്ത്രിക്കാൻ സംഭാഷണങ്ങൾ സ്വയം സംഗ്രഹിക്കുക
- **ഏജന്റ് സ്ക്രാച്ച്പാഡ്**: ഉപയോക്തൃ ഇഷ്ടങ്ങളും പൂർത്തിയാക്കിയ പ്രവർത്തനങ്ങൾക്കുള്ള സ്ഥിരമായ മെമ്മറി നടപ്പിലാക്കുക
- **ടോക്കൺ മാനേജ്മെന്റ്**: ദീർഘമായ സംഭാഷണങ്ങളിൽ ടോക്കൺ ഉപയോഗം നിരീക്ഷിക്കുകയും മെച്ചപ്പെടുത്തുകയും ചെയ്യുക
- **കോൺടെക്സ്റ്റ് സംരക്ഷണം**: സംഭാഷണ ചുരുക്കലുകളിൽ പ്രധാന വിവരങ്ങൾ നിലനിർത്തുക

## യഥാർത്ഥ ലോക ഉപയോഗങ്ങൾ:
- **കസ്റ്റമർ സർവീസ് ബോട്ടുകൾ**: ഉപഭോക്തൃ ഇഷ്ടങ്ങൾ സെഷനുകൾക്കിടയിൽ ഓർക്കുക
- **വ്യക്തിഗത സഹായികൾ**: പുരോഗമിക്കുന്ന പ്രോജക്റ്റുകളും ഉപയോക്തൃ ശീലങ്ങളും നിരീക്ഷിക്കുക
- **വിദ്യാഭ്യാസ ട്യൂട്ടറുകൾ**: വിദ്യാർത്ഥികളുടെ പുരോഗതിയും പഠന ഇഷ്ടങ്ങളും നിലനിർത്തുക
- **ആരോഗ്യ സഹായികൾ**: ടോക്കൺ പരിധി മാനിച്ച് രോഗിയുടെ ചരിത്രം സൂക്ഷിക്കുക

## അടുത്ത ഘട്ടങ്ങൾ:
- കൂടുതൽ സങ്കീർണ്ണമായ സ്ക്രാച്ച്പാഡ് സ്കീമകൾ നടപ്പിലാക്കുക
- മൾട്ടി-യൂസർ സാഹചര്യങ്ങൾക്ക് ഡാറ്റാബേസ് സംഭരണം ചേർക്കുക
- ഡൊമെയ്ൻ-സ്പെസിഫിക് ആവശ്യങ്ങൾക്ക് കസ്റ്റം ചുരുക്കൽ തന്ത്രങ്ങൾ സൃഷ്ടിക്കുക
- സെമാന്റിക് മെമ്മറി തിരയലിനായി വെക്റ്റർ ഡാറ്റാബേസുകളുമായി സംയോജിപ്പിക്കുക
- പൂർണ്ണ കോൺടെക്സ്റ്റോടെ ദിവസങ്ങൾക്കുശേഷം സംഭാഷണങ്ങൾ പുനരാരംഭിക്കാൻ കഴിയുന്ന ഏജന്റുമാരെ നിർമ്മിക്കുക


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖ പ്രാമാണികമായ ഉറവിടമായി പരിഗണിക്കണം. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
